In [0]:
customers = spark.table("customer_transactions")
customers.show(5)


+-----------+---+------+---------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+
|customer_id|age|gender|  country|annual_income|spending_score|num_purchases|avg_purchase_value|membership_years|website_visits_per_month|cart_abandon_rate|churned|       feedback_text|last_purchase_date|
+-----------+---+------+---------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+
|          1| 37|  Male|  Germany|        85886|            14|           18|              41.2|               6|                      20|             0.95|      0|Very satisfied wi...|        2025-06-22|
|          2| 40|  Male|    India|        41041|             4|           10|             31.73|               4|                      29|             0.21|      0|Good quality and

In [0]:
customers.printSchema()


root
 |-- customer_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)
 |-- annual_income: long (nullable = true)
 |-- spending_score: long (nullable = true)
 |-- num_purchases: long (nullable = true)
 |-- avg_purchase_value: double (nullable = true)
 |-- membership_years: long (nullable = true)
 |-- website_visits_per_month: long (nullable = true)
 |-- cart_abandon_rate: double (nullable = true)
 |-- churned: long (nullable = true)
 |-- feedback_text: string (nullable = true)
 |-- last_purchase_date: date (nullable = true)



In [0]:
delta_path = "dbfs:/FileStore/delta/customer_transactions"

In [0]:
customers.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("customer_transactions_delta")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS customer_transactions_delta_sql
USING DELTA
AS
SELECT * FROM customer_transactions_delta;


num_affected_rows,num_inserted_rows


In [0]:
spark.sql("SELECT COUNT(*) FROM customer_transactions").show()


+--------+
|COUNT(*)|
+--------+
|   10000|
+--------+



In [0]:
try:
    wrong_schema = spark.createDataFrame(
        [("a", "b", "c")],
        ["x", "y", "z"]
    )

    wrong_schema.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("customer_transactions_delta")

except Exception as e:
    print("Schema enforcement triggered correctly")
    print(e)


Schema enforcement triggered correctly
[_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 9205a59f-8d71-4e07-bbf1-0df6711d274a).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- customer_id: long (nullable = true)
-- age: long (nullable = true)
-- gender: string (nullable = true)
-- country: string (nullable = true)
-- annual_income: long (nullable = true)
-- spending_score: long (nullable = true)
-- num_purchases: long (nullable = true)
-- avg_purchase_value: double (nullable = true)
-- membership_years: long (nullable = true)
-- website_visits_per_month: long (nullable = true)
-- cart_abandon_rate: double (nullable = true)
-- churned: long (nullable = true)
-- feedback_text: s